In [1]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

import pandas as pd
import numpy as np

from pathlib import Path
from datetime import datetime
import multiprocessing
import logging

cores = multiprocessing.cpu_count()
print(f'cores: {cores}')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

cores: 4


In [2]:
path_data = Path('../../data/corpora/clean_tutby_126784.csv')
# path_data = Path('../../data/corpora/clean_tutby_126784_header.csv')

path_d2v = Path('../../data/model/d2v/model_d2v_clean_tutby_126784.model')
# path_d2v = Path('../../data/model/d2v/model_d2v_clean_tutby_126784_header.model')

path_embedding = Path('../../data/emb/emb_clean_tutby_126784_d2v.npy')
# path_embedding = Path('../../data/emb/emb_clean_tutby_126784_header_d2v.npy')

column_name = 'document'
# column_name = 'header'

In [3]:
data = pd.read_csv(path_data)

documents = data[column_name]
documents = documents.fillna('')

print(data.shape)
data.head(5)

(126784, 6)


,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,главный тренер солигорский шахтер олег кубарев...,['футбол']
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,план восстановление рисунок пока художник илья...,"['архитектура', 'живопись', 'ЖКХ']"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,областной управление мчс день пожарный служба ...,['министерства']
3,https://news.tut.by/550309.html,Футбол,Станислав Драгун дебютировал за БАТЭ в матче с...,2017-07-06T22:11:00+03:00,чемпион беларусь бате воспользоваться пауза че...,"['футбол', 'БАТЭ']"
4,https://news.tut.by/550310.html,В мире,Генпрокурор Украины пообещал открыть уголовное...,2017-07-06T22:28:00+03:00,генпрокуратура украина открывать уголовный про...,"['Ситуация в Украине', 'государственные перево..."


In [4]:
%%time
print(datetime.now())

params_doc2vec = {
    'dm': 0,
    'vector_size': 300,
    'window': 15,
    'min_count': 20,
    'sample': 1e-5,
    'workers': cores,
    'epochs': 40,
    'hs': 0,
    'negative': 5,
    'dbow_words': 1,
}

documents_gensim = [TaggedDocument(document.split(), [index]) for index, document in documents.items()]

model = Doc2Vec(documents=documents_gensim, **params_doc2vec)

model.save(str(path_d2v))
model = Doc2Vec.load(str(path_d2v))

embeddings = model.docvecs.vectors_docs

with open(path_embedding, 'wb') as file:
    np.save(file, embeddings)

with open(path_embedding, 'rb') as file:
    embeddings = np.load(file)
    
print(embeddings.shape)

2020-11-15 00:44:02.203523


2020-11-15 00:44:09,487 : INFO : collecting all words and their counts
2020-11-15 00:44:09,492 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-11-15 00:44:10,506 : INFO : PROGRESS: at example #10000, processed 2466256 words (2432683/s), 99885 word types, 10000 tags
2020-11-15 00:44:11,537 : INFO : PROGRESS: at example #20000, processed 5035139 words (2494384/s), 143097 word types, 20000 tags
2020-11-15 00:44:12,721 : INFO : PROGRESS: at example #30000, processed 7623391 words (2189747/s), 176652 word types, 30000 tags
2020-11-15 00:44:13,500 : INFO : PROGRESS: at example #40000, processed 10215111 words (3331902/s), 205336 word types, 40000 tags
2020-11-15 00:44:14,199 : INFO : PROGRESS: at example #50000, processed 12787803 words (3684520/s), 231745 word types, 50000 tags
2020-11-15 00:44:14,809 : INFO : PROGRESS: at example #60000, processed 15423913 words (4327137/s), 256913 word types, 60000 tags
2020-11-15 00:44:15,415 : INFO : PROGRESS: at exa

2020-11-15 00:45:47,218 : INFO : EPOCH 1 - PROGRESS: at 20.68% examples, 48753 words/s, in_qsize 8, out_qsize 0
2020-11-15 00:45:48,286 : INFO : EPOCH 1 - PROGRESS: at 21.10% examples, 48799 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:45:49,447 : INFO : EPOCH 1 - PROGRESS: at 21.59% examples, 48888 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:45:50,498 : INFO : EPOCH 1 - PROGRESS: at 21.94% examples, 48796 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:45:51,720 : INFO : EPOCH 1 - PROGRESS: at 22.27% examples, 48484 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:45:52,769 : INFO : EPOCH 1 - PROGRESS: at 22.57% examples, 48334 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:45:53,903 : INFO : EPOCH 1 - PROGRESS: at 22.99% examples, 48247 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:45:54,979 : INFO : EPOCH 1 - PROGRESS: at 23.31% examples, 48144 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:45:55,995 : INFO : EPOCH 1 - PROGRESS: at 23.59% examples, 47973 words/s, in_qsize 7, out_

2020-11-15 00:47:07,242 : INFO : EPOCH 1 - PROGRESS: at 49.06% examples, 47180 words/s, in_qsize 8, out_qsize 0
2020-11-15 00:47:08,327 : INFO : EPOCH 1 - PROGRESS: at 49.53% examples, 47257 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:47:09,392 : INFO : EPOCH 1 - PROGRESS: at 50.00% examples, 47322 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:47:10,427 : INFO : EPOCH 1 - PROGRESS: at 50.43% examples, 47393 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:47:11,475 : INFO : EPOCH 1 - PROGRESS: at 50.88% examples, 47448 words/s, in_qsize 8, out_qsize 0
2020-11-15 00:47:12,484 : INFO : EPOCH 1 - PROGRESS: at 51.30% examples, 47491 words/s, in_qsize 6, out_qsize 1
2020-11-15 00:47:13,550 : INFO : EPOCH 1 - PROGRESS: at 51.75% examples, 47565 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:47:14,567 : INFO : EPOCH 1 - PROGRESS: at 52.18% examples, 47627 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:47:15,576 : INFO : EPOCH 1 - PROGRESS: at 52.62% examples, 47673 words/s, in_qsize 7, out_

2020-11-15 00:48:25,626 : INFO : EPOCH 1 - PROGRESS: at 76.61% examples, 46756 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:48:26,758 : INFO : EPOCH 1 - PROGRESS: at 76.76% examples, 46618 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:48:27,783 : INFO : EPOCH 1 - PROGRESS: at 76.93% examples, 46518 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:48:28,981 : INFO : EPOCH 1 - PROGRESS: at 77.15% examples, 46385 words/s, in_qsize 8, out_qsize 0
2020-11-15 00:48:30,051 : INFO : EPOCH 1 - PROGRESS: at 77.38% examples, 46309 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:48:31,080 : INFO : EPOCH 1 - PROGRESS: at 77.65% examples, 46267 words/s, in_qsize 8, out_qsize 0
2020-11-15 00:48:32,194 : INFO : EPOCH 1 - PROGRESS: at 77.90% examples, 46184 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:48:33,289 : INFO : EPOCH 1 - PROGRESS: at 78.25% examples, 46145 words/s, in_qsize 8, out_qsize 0
2020-11-15 00:48:34,408 : INFO : EPOCH 1 - PROGRESS: at 78.59% examples, 46121 words/s, in_qsize 7, out_

...

2020-11-15 03:23:46,187 : INFO : EPOCH 40 - PROGRESS: at 10.52% examples, 52775 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:23:47,266 : INFO : EPOCH 40 - PROGRESS: at 10.96% examples, 52944 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:23:48,300 : INFO : EPOCH 40 - PROGRESS: at 11.46% examples, 53177 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:23:49,355 : INFO : EPOCH 40 - PROGRESS: at 11.92% examples, 53255 words/s, in_qsize 8, out_qsize 0
2020-11-15 03:23:50,384 : INFO : EPOCH 40 - PROGRESS: at 12.34% examples, 53321 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:23:51,440 : INFO : EPOCH 40 - PROGRESS: at 12.76% examples, 53209 words/s, in_qsize 8, out_qsize 0
2020-11-15 03:23:52,519 : INFO : EPOCH 40 - PROGRESS: at 13.21% examples, 53290 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:23:53,579 : INFO : EPOCH 40 - PROGRESS: at 13.70% examples, 53434 words/s, in_qsize 8, out_qsize 0
2020-11-15 03:23:54,605 : INFO : EPOCH 40 - PROGRESS: at 14.17% examples, 53525 words/s, in_qsiz

2020-11-15 03:25:03,495 : INFO : EPOCH 40 - PROGRESS: at 44.16% examples, 55405 words/s, in_qsize 7, out_qsize 1
2020-11-15 03:25:04,526 : INFO : EPOCH 40 - PROGRESS: at 44.60% examples, 55435 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:25:05,538 : INFO : EPOCH 40 - PROGRESS: at 45.04% examples, 55410 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:25:06,570 : INFO : EPOCH 40 - PROGRESS: at 45.48% examples, 55452 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:25:07,676 : INFO : EPOCH 40 - PROGRESS: at 45.91% examples, 55445 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:25:08,686 : INFO : EPOCH 40 - PROGRESS: at 46.33% examples, 55454 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:25:09,746 : INFO : EPOCH 40 - PROGRESS: at 46.76% examples, 55481 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:25:10,802 : INFO : EPOCH 40 - PROGRESS: at 47.29% examples, 55499 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:25:11,855 : INFO : EPOCH 40 - PROGRESS: at 47.75% examples, 55538 words/s, in_qsiz

2020-11-15 03:26:20,499 : INFO : EPOCH 40 - PROGRESS: at 77.23% examples, 55887 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:26:21,613 : INFO : EPOCH 40 - PROGRESS: at 77.67% examples, 55877 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:26:22,665 : INFO : EPOCH 40 - PROGRESS: at 78.15% examples, 55887 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:26:23,720 : INFO : EPOCH 40 - PROGRESS: at 78.64% examples, 55900 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:26:24,724 : INFO : EPOCH 40 - PROGRESS: at 79.10% examples, 55907 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:26:25,792 : INFO : EPOCH 40 - PROGRESS: at 79.56% examples, 55911 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:26:26,908 : INFO : EPOCH 40 - PROGRESS: at 80.02% examples, 55899 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:26:27,974 : INFO : EPOCH 40 - PROGRESS: at 80.43% examples, 55909 words/s, in_qsize 7, out_qsize 0
2020-11-15 03:26:29,017 : INFO : EPOCH 40 - PROGRESS: at 80.87% examples, 55922 words/s, in_qsiz

2020-11-15 03:27:16,539 : INFO : loaded ..\..\data\model\d2v\model_d2v_clean_tutby_126784.model


(126784, 300)
Wall time: 2h 43min 15s
